# Tile Data

## Imports

In [ ]:
import os

from tlib import tgeo, tutils
from tlib.tgeo import tgdal
from asos import settings

%load_ext autoreload
%autoreload 2

## Get Files

In [ ]:
# get all files from given mask folder
files = tutils.files.files_from_folder(
    folder=os.path.join(settings.data_folder_raw, 'mask'),
    whole_path=True,
    regex_filter='.tif'
)
print(len(files))

## Find Tile Positions

In [ ]:
def get_mask(file, wdpa):    
    mask = tgeo.geotif.get_array(file=file, transpose=True)
    mask = mask.astype(bool)  # masks have values 0, 1, 2 and 3; with this operation 0 becomes False and the other values become True
    return mask if wdpa else ~mask


for wdpa in [True, False]:
    tgeo.tiling.run_tiling(
        files=files,
        output_folder=tutils.files.join_paths(settings.working_folder, str(wdpa)),
        tile_dims=(256, 256),
        overlap=None,
        step_size=8,
        get_mask=lambda file: get_mask(file=file, wdpa=wdpa),
    )

## GDAL Tiling

In [ ]:
output_folder = '/media/timo/My Book/mapinwild_tiles'

for wdpa in ['True', 'False']:
    tgdal.utils.tile_using_tile_positions(
        tile_positions_file=os.path.join(settings.working_folder, wdpa, 'tile_positions.npy'),
        image_folder_names=['ESA_WC', 'mask', 'NIIR', 's2_autumn', 's2_spring', 's2_summer', 's2_winter', 'S1'],
        output_folder=tutils.files.join_paths(output_folder, wdpa),
    )